# ARIM-Academy：　機器データ利活用ユースケース
### タイトル：機械学習によるTi合金の疲労・クリープ寿命予測
### 機器：クリープ試験機
### 分析：回帰分析（勾配ブースティング回帰、SHAP評価）


## データセット

<div style="border:1px solid #000; padding:10px;">
    
本編で扱う creep_data.xlsx" は、177種のTi合金にかかるクリープデータセットです。

[1] S. Sucheta, R. Ashish and S. A. Kumar, "Machine learning assisted interpretation of creep and fatigue life in titanium alloys", *APL Machine Learning*, **1**, 016102 (2023)

"creep_data.xlsx" には、合金の組成のほか、クリープの実験条件ならびに熱処理のデータがセットで含まれています。

<br>  
<img src="./img/m_016102_1_f1.jpeg" width="50%"><br> 

---



**Chemical composition** 
1. Titanium (wt %) Ti 
1. Alumunium (wt %) Al 
1. Vanadium (wt %) Vn 
1. Carbon (wt %) C 
1. Nitrogen (wt %) N 
1. Oxygen (wt %) O 
1. Hydrogen (wt %) H 
1. Iron (wt %) Fe 0 
1. Silicon (wt %) Si 
1. Tin (wt %) Sn 
1. Niobium (wt %) Nb 
1. Molybednum (wt %) Mo 
1. Zirconium (wt %) Zr 
1. Boron (wt %) B 
1. Chromium (wt %) Cr 


**Experimental parameters**  
1. Rupture strain (%) ϵr 
1. Temperature of measurement (cel) Tcreep 
1. Steady state strain rate (1/s) ˙ ϵ 
1. Applied stress (MPa) σ 

**Heat treatment conditions**  
1. Aging temperature (cel) Tage 
1. Aging time (hrs) tage 
1. Solution temperature (cel) Tsol
1. Solution time (cel) tsol

### 教材への接続
google colabにおけるオンラインの場合にこのラインを実行します。（<font color="red">Google colabに接続しない場合には不要</font>）

In [ ]:
!pip install pycaret
!git clone https://github.com/ARIM-Usecase/Example_1.git
%cd Example_1

### ライブラリのインポート
コード実行で必要なpythonのライブラリをimport文でロードします。

In [2]:
# 汎用ライブラリ
import pandas as pd

from pycaret.regression import *

import warnings
warnings.filterwarnings('ignore')

### サンプルファイルの読み込み
pandasライブラリのread_csv()関数は、csvファイルを読み込んでpandasのDataFrame形式に変換する関数です。ここでは[data]フォルダーにあるcreep_pycaret.csvファイルをDataFrameとして読み込み、その結果をdfという変数に格納します。

In [3]:
df = pd.read_csv('./data/creep_pycaret.csv')    # database with standardized features and log-scaled rupture life
df

,Rupture time (hour),Al,V,Fe,Mo,N,Solution treated temp(cel),Anneal temp (cel),Annealing Time (hour),Temperature of creep test (cel),Stress (Mpa),steady state strain rate (1/s),Strain to rupture (%) (Efc)
0,-1.932929,6.61,4.230000,1.180000,0.657175,0.011,403.559322,551.169492,5.009266,600,319.0,3.240000e-06,14.758023
1,-1.515700,6.61,4.230000,1.180000,0.657175,0.011,403.559322,551.169492,5.009266,600,250.0,4.400000e-05,14.758023
2,-1.335358,6.61,4.230000,1.180000,0.657175,0.011,403.559322,551.169492,5.009266,500,520.0,4.430000e-05,14.758023
3,-1.045757,6.00,4.000000,0.400000,0.657175,0.050,403.559322,551.169492,5.009266,600,319.0,3.520000e-04,26.580000
4,-1.018634,6.61,4.230000,1.180000,0.657175,0.011,403.559322,551.169492,5.009266,600,222.0,2.550000e-05,14.758023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2.875061,6.00,4.000000,0.250000,0.657175,0.050,403.559322,900.000000,2.000000,475,300.0,2.550000e-08,14.220000
122,2.875061,6.20,3.027062,0.386475,2.000000,0.004,1050.000000,550.000000,24.000000,520,310.0,1.000000e-05,14.758023
123,3.064458,6.20,3.027062,0.386475,2.000000,0.004,900.000000,580.000000,8.000000,520,310.0,1.200000e-04,14.758023
124,3.209247,6.51,4.080000,0.160000,0.657175,0.005,403.559322,940.000000,4.000000,455,379.0,9.160000e-07,14.758023


### pycaretを使ったモデル選択
PyCaretを使用して回帰モデルのパイプラインを設定し、指定されたモデルの中からRMSE（Root Mean Squared Error）に基づいて最も優れたモデルを選択するプロセスを実行します。まず、`setup()`メソッドを呼び出すと、データが自動的にクリーニングされ、トレーニング用とテスト用に分割されます。ここで、各種の前処理（エンコーディング、スケーリング、欠損値補完など）が行われます。

In [4]:
# setup the dataset
grid = setup(data=df, 
             target='Rupture time (hour)',
             train_size=0.9, 
             session_id =234
             )

,Description,Value
0,Session id,234
1,Target,Rupture time (hour)
2,Target type,Regression
3,Original data shape,"(126, 13)"
4,Transformed data shape,"(126, 13)"
5,Transformed train set shape,"(113, 13)"
6,Transformed test set shape,"(13, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


### モデルの比較と最良モデルの選択
`compare_models()`メソッドで指定したモデル（include=models）を全てトレーニングし、最も良いパフォーマンスを示したモデル（ここではRMSEが最小のモデル）が best 変数に格納されます。なお、ここでは比較するmodelはmodelsでリスト化している、下記の回帰モデルです。
* 'lasso': Lasso回帰 (L1正則化)
* 'ridge': Ridge回帰 (L2正則化)
* 'svm': サポートベクターマシン
* 'knn': k-Nearest Neighbors
* 'dt': 決定木回帰
* 'rf': ランダムフォレスト回帰
* 'gbr': 勾配ブースティング回帰

In [4]:
models = [ 'lasso', 'ridge', 'svm', 'knn', 'dt', 'rf',  'gbr']
best = compare_models(sort='RMSE', include=models)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.4130,0.3179,0.5476,0.7946,0.2253,0.6534,0.0250
rf,Random Forest Regressor,0.4430,0.4033,0.6056,0.7372,0.2415,0.6470,0.0590
dt,Decision Tree Regressor,0.4464,0.4363,0.6309,0.7120,0.2521,0.6916,0.0090
knn,K Neighbors Regressor,0.6609,0.7042,0.8318,0.5241,0.3219,0.9742,0.0120
ridge,Ridge Regression,0.7527,0.9034,0.9445,0.3762,0.3793,1.1158,0.0090
svm,Support Vector Regression,0.8003,1.0017,0.9887,0.3388,0.4001,1.1393,0.0090
lasso,Lasso Regression,0.8388,1.1047,1.0345,0.2916,0.3928,1.1462,0.4720


【解説】

sort='RMSE': モデルの性能を評価する指標として、RMSE（Root Mean Squared Error）を基準にしています。RMSEが低いほど、モデルの予測精度が高いことを示します。  
include=models: 比較に含めるモデルを指定しています。前述の models リストにある7つのモデルが含まれています。